# Формульные конструкции

* Цель проекта - автоматизированный (непредзаданный) поиск повторяющихся конструкций с той или иной степенью вариативности.
* Данные - Корпус древнеисландских саг
* Объем корпуса - 1.5 млн токенов
* Разметка - словоизменительный тип и лемма

Работа состоит из следующих этапов:

    1) Формирование списка нграмм
    2) Фильтрация на основе лингвистических особенностей исландского языка
    3) «Схлопывание» контекстных вариантов
    4) Кластеризация
    5) Создание базы данных
    6) Создание сайта

В настоящем ноутбуке представлен код для реализации 1-5 этапов проекта

In [1]:
import os
import json
import copy
from collections import Counter, defaultdict, namedtuple
import sqlite3

from tqdm.auto import tqdm
import pandas as pd
import numpy as np
# import pickle as pkl

from bs4 import BeautifulSoup
from nltk.util import ngrams 

from scipy.cluster.hierarchy import complete, dendrogram, fcluster
from gensim.models.fasttext import FastText, load_facebook_vectors
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
db = sqlite3.connect("iceform.db")
cur = db.cursor()

## 0. Планирование и архитектура

Глобальные переменные:
    
- настройки языковые - json-файл
- путь к файлам
- папка для пиклов


Классы:

- Node Word- слова
- Ngram: набор слов 

Словари:
- координаты
- частотность
=> в базу вместе с нграммой

In [3]:
cur.execute("""
CREATE TABLE text (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    text_name TEXT,
    file_name TEXT
)""")

cur.execute("""
CREATE TABLE token (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    wordform TEXT,
    morph TEXT,
    pos TEXT,
    lemma TEXT,
    ngram_text_id INT
)""")

cur.execute("""
CREATE TABLE text_content (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    text_id INT,
    sentence_id INT,
    idx INT,
    token_id INT
)""")

cur.execute("""
CREATE TABLE ngram_item (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    representation TEXT,
    lemma TEXT,
    pos TEXT,
    kind TEXT
)""")

cur.execute("""
CREATE TABLE ngram_content (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    ngram_id INT,
    ngram_item_id INT,
    token_id INT,
    keep BOOL
)""")

cur.execute("""
CREATE TABLE ngram_coords (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    unique_ngram_id INT,
    text_id INT,
    sentence_id INT,
    start INT,
    end INT,
    short_ngram_id INT
);""")

cur.execute("""
CREATE TABLE short_ngrams (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    n_occurrences INT,
    representation INT
)""")

cur.execute("""
CREATE TABLE clustering (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    ngram_id INT,
    short_ngram_id INT,
    cluster_idx INT
)""")

cur.execute("""
CREATE TABLE unique_ngrams (
    id INTEGER PRIMARY KEY AUTOINCREMENT, 
    n_occurrences INT, 
    short_ngram_id INT
)
""")

db.commit()

## 1. Парсинг корпуса


<b>Задача:</b> распознавание файлов и запись в базу

### 1.1. Обработка расшифровок морфологических тегов

Корпус саг состоит из: 
   * 49 xml-файлов, содрержащих тексты саг в TEI <br> 
      Каждое слово в корпусе сопровождается леммой и морфологическим разбором: <br> 
      Пример: ```<w lemma="Þorgerður" type="nven-s">Þorgerður</w>``` 
  
  
  
   
   * Файл sagHdr.xml, содержащий расшифровку морфологических тегов<br> 
      Пример: ```<item id="nven-s" occurs="5484">noun feminie singular nominative proper-noun</item>```

Для удобства дальнейшей работы из расшифровок морфологических тегов были автоматически выделены наиболее значимые для анализа характеристики и записанные в словарь, содержащий следующие ключи: 
   * pos - часть речи
   * gram - грамматические характеристики слов (на данном этапе учитывается только падеж у существительных и прилагательных)
   * dep - характеристики зависимых или главных слов, необходимых для соблюдения правил согласования и подчинения 
   
Далее каждому тегу был сопоставлен соответвующий разбор (CODE_POS).

In [4]:
PATH_codes = "/home/dkbrz/github/IceForm/xml/sagHdr.xml"
PATH_texts = "/home/dkbrz/github/IceForm/xml"
PATH_deps = "/home/dkbrz/github/IceForm/2020-06-21/dependencies.json"
PATH_gram = "/home/dkbrz/github/IceForm/2020-06-21/gram_info.json"

In [5]:
class GramInfo:
    
    """
    GramInfo is a class that:
        1) stores descriptions of morphological tags used for saga annotation
        2) stores a system of abbreviations, rules, and weights for easy preprocessing
        3) provides methods for preprocessing morphological tags descriptions
    """
    
    def __init__(self, 
                 path_deps="dependencies.json", 
                 path_codes="xml/sagHdr.xml", 
                 path_gram="gram_info.json"):
        self.path_deps = path_deps
        self.path_codes = path_codes
        self.path_gram = path_gram
        self.depend = self.add_depend()
        self.morph_info = self.add_morph_info()
        self.add_gram_info()
    
    def add_morph_info(self):
        """
        sagHdr.xml contains morphological tags and their descriptions
        """
        morph_info = self.open_xml(self.path_codes)
        morph_info = morph_info.find_all("list")[27]
        morph_info = [i for i in morph_info if i != '\n']
        return morph_info
        
    def add_depend(self):
        """
        dependencies.json contains morphological dependencies
        """
        return self.open_json(self.path_deps)
        
    def add_gram_info(self):
        """
        gram_info.json contains morphological inforamtion 
        Categories:
            POS_short - short name of pos
            POS_values - weight of pos
            gram_short - short name of certain grammatical information
            case_short - short name of case
            case_values - weight of case
        """
        gram_info = self.open_json(self.path_gram)
        self.POS_short = gram_info['POS_short']
        self.POS_values = gram_info['POS_values']
        self.gram_short = gram_info['gram_short']
        self.case_short = gram_info['case_short']
        self.case_values = gram_info['case_values']
        
    def get_value(self, morph, attr):
        """
        This function finds abbreviations for morphological tags
        """
        for name in morph:
            val = self.__dict__[attr].get(name)
            if val: return val
        return None
    
    def set_dependencies(self, pos, name, morph_dict):
        """
        This function sets dependencies based on a morphological tag
        """
        if pos == 'CINF': dep = 'Inf'
        elif pos == 'Inf': dep = 'CINF'
        else: dep = self.get_value(morph, 'depend')
        if dep: morph_dict['dep'] = dep   
    
    def set_case(self, morph, morph_dict):
        """
        This function sets case based on a morphological tag
        """
        case = self.get_value(morph, 'case_short')
        if case: morph_dict['gram'] = case
         
    def set_pos(self, morph, morph_dict):
        """
        This function sets POS based on a morphological tag
        """
        pos = self.POS_short.get(morph[0])
        if pos in ('V', 'PRO', 'C', 'N') and len(morph) > 1:
            alter_pos = self.get_value(morph, 'gram_short')
            if alter_pos: pos = alter_pos
        morph_dict['pos'] = pos 
        return pos
    
    @staticmethod
    def open_json(path):
        """
        This function opens json file and
        converts it to BeautifulSoup for an easy data extraction process 
        """
        with open(path) as json_file:
            data = json.load(json_file)
        return data
    
    @staticmethod
    def open_xml(path):
        """
        This function opens xml file 
        """
        with open(path, 'r', encoding='utf-8') as f:
            xml = f.read()
            data = BeautifulSoup(xml, 'lxml')
        return data

In [6]:
grinfo = GramInfo(path_codes=PATH_codes, path_deps=PATH_deps, path_gram=PATH_gram)

In [7]:
CODE_POS = defaultdict(dict)

deps_pos = {'PREP', 'ART', 'CINF', 'Adj', 'Inf', 'PRO'}
case_pos = {'N', 'Adj', 'PPro', 'Prp'}

for morph_line in grinfo.morph_info:
    
    id_ = morph_line.attrs['id']
    morph = morph_line.string.split(' ')
    pos = grinfo.set_pos(morph, CODE_POS[id_])
    
    if pos in deps_pos:
        grinfo.set_dependencies(pos, morph, CODE_POS[id_])
    
    if pos in case_pos:
        grinfo.set_case(morph, CODE_POS[id_])

CODE_POS['pun']['pos'] = 'pun'

### 1.2. Парсинг текстов саг

Каждый текст в корпусе был распаршен и записан в базу. По мере обработки тектов саг, для каждого токена была сохранена информация о его лемме, словоформе и морфологической информации, полученной на предудщем этапе. 

In [8]:
class Token:
    
    def __init__(self, wordform, morph, pos, lemma):
        self.idx = None
        self.wordform = wordform
        self.morph = morph
        self.pos = pos
        self.lemma = lemma
    
    def __eq__(self, other):
        if other.wordform != self.wordform:
            return False
        if other.morph != self.morph:
            return False 
        return True

TextContent = namedtuple("TextContent", ["text_id", "sentence_id", "w_idx", "token_id"])

class TokenStorage:
    
    def __init__(self):
        self.tdict = {}
        self.tlist = []
        self.tidx = 0
        
    def get_id(self, wordform, morph):
        pair = (wordform, morph)
        if pair in self.tdict:
            return self.tdict[pair], True
        else:
            self.tdict[pair] = self.tidx
            self.tidx += 1
            return self.tdict[pair], False
    
    def add_token(self, token):
        token.idx = self.tdict[(token.wordform, token.morph)] 
        self.tlist.append(token)
        
    def to_db(self):
        start = min(self.tdict.values())
        assert start == 0
        data = [(t.idx, t.wordform, t.morph, t.pos, t.lemma) for t in self.tlist]
        cur.executemany("""
        INSERT INTO token (
            id, wordform, morph, pos, lemma
        ) VALUES (?, ?, ?, ?, ?)""", data)
        db.commit()

In [9]:
class Text:
    
    def __init__(self, idx, fname):
        self.idx = idx
        self.fname = fname
        self.soup = self.read_file()
        self.title = self.find_title()
        self.text_content = self.parse_text()
        
    def read_file(self):
        path = os.path.join(PATH_texts, self.fname)
        with open(path, 'r', encoding='utf-8') as f:
            soup = BeautifulSoup(f.read(), 'lxml')
        return soup
    
    def find_title(self):
        return self.soup.find("title").get_text()
    
    def parse_text(self):
        global t_storage
        
        text_content = []
        
        for s_id, sentence in enumerate(self.soup.find_all("s")):
            w_idx = 0
            for word in sentence:
                is_word = word.name == 'w' and word.attrs['lemma'].lower() != 'at'
                is_punct = word.name == 'c' and s_id != (len(sentence) - 2)                
                if not (is_word or is_punct):
                    continue
                wordform = word.text.lower()
                if is_word:
                    gram = word.attrs['type']
                else:
                    gram = 'pun'
                idx, known = t_storage.get_id(wordform, gram)
                if not known:
                    pos = CODE_POS.get(gram, {}).get("pos")
                    token = Token(wordform=wordform, morph=gram, pos=pos, lemma=wordform)
                    t_storage.add_token(token)
                
                t_word = TextContent(text_id=self.idx, sentence_id=s_id, w_idx=w_idx, token_id=idx)
                text_content.append(t_word)
                w_idx += 1
        return text_content
    
    def to_db(self):
        cur.execute(
            "INSERT INTO text VALUES (?, ?, ?)", 
            (self.idx, self.title, self.fname))
        cur.executemany("""
        INSERT INTO text_content (
            text_id, sentence_id, idx, token_id
        ) VALUES (?, ?, ?, ?)""", self.text_content)
        db.commit()

In [10]:
t_storage = TokenStorage()

In [11]:
text_list = sorted([
    f for f in os.listdir(PATH_texts) 
    if not f.startswith("sagHdr")
])

In [12]:
for idx, fname in tqdm(enumerate(text_list), leave=False, total=len(text_list)):
    text = Text(idx, fname)
    text.to_db()

t_storage.to_db()

### 1.3. Более абстрактные представления

В связи с особенностями исландского языка (омонимия, словоизменение, отсутвие фиксированного порядка слов), работать только с леммами слов затруднительно, так как на этапе "схлопывания" нграмм есть риск объединить разные по смыслу конструкции. Поэтому слова в корпусе были представленны в следующих видах в зависимости от частеричной принадлежности и ее значимости в формировании смысла конструкции:

   * <b>Часть речи</b> — для артиклей, междометий, местоимений и неизвестных слов 
   * <b>Часть речи + Падеж</b> — для имен собственных и прилагательных
   * <b>Часть речи + Падеж + "_" + Лемма</b> — для существительных
   * <b>Часть речи + "_" + Лемма</b> — для глаголов, наречий, союзов и предлогов   

In [13]:
pos_skip = {'ART', 'Adj', 'EX', 'UNK' , 'PPro', 'PRO', 'Prp'}

In [14]:
def get_representation(morph, pos, lemma):
    if pos is None:
        pos = ""
    gram = CODE_POS.get(morph, {}).get('gram', "")
    ginf = pos + gram
    if pos in pos_skip: 
        return ginf, pos, None, ginf
    return ginf, pos, lemma, f"{ginf}_{lemma}"

In [15]:
cur.execute("SELECT id, morph, pos, lemma FROM token")
tokens = cur.fetchall()

In [16]:
ngram_items_dict = {}
ngram_items_list = []
token_update = []
ngram_items_idx = 0

for idx, morph, pos, lemma in tokens:
    ginf, pos, lemma, rprs =  get_representation(morph, pos, lemma)
    if rprs in ngram_items_dict:
        token_update.append((ngram_items_dict[rprs], idx))
    else:
        ngram_items_dict[rprs] = ngram_items_idx
        token_update.append((ngram_items_dict[rprs], idx))
        ngram_items_list.append((ngram_items_dict[rprs], pos, lemma, rprs, ginf))
        ngram_items_idx += 1

In [17]:
cur.executemany("INSERT INTO ngram_item (id, pos, lemma, representation, kind) VALUES (?, ?, ?, ?, ?)", ngram_items_list)
db.commit()

In [18]:
cur.executemany("UPDATE token SET ngram_text_id = ? WHERE id = ?", token_update)
db.commit()

## 2. Формирование списка нграмм

<b>Задача</b>: формирование списка нграмм на основе корпуса 

### 2.1. Формирование списка предложений корпуса

Для удобства процесса формирования нграмм, был составлен список предложений всего корпуса, в котором каждое слово сопровождается не только грамматической информацией, но и его расположение в тексте (координаты). Сохранение координат необходимо для осуществления поиска нграмм в текстах, которое будет реализовано на сайте. 

In [19]:
NgramCoord = namedtuple("NgramCoord", ["text_index", "sent_index", "word_start", "word_end"])

def to_ngram_coord(words):
    text_index = min([i.text_index for i in words])
    sent_index = min([i.sent_index for i in words])
    word_start = min([i.word_index for i in words])
    word_end = max([i.word_index for i in words])
    return NgramCoord(text_index, sent_index, word_start, word_end)
                

class AllNgrams:
    
    def __init__(self):
        self.freq = defaultdict(int)
        self.coord = defaultdict(set)
    
    def add(self, words):
        coords = to_ngram_coord(words)
        self.coord[words].add(coords)
        self.freq[words] += 1

In [20]:
class Word:
    
    def __init__(self, lemma, morph, show, text_index, sent_index, word_index):
        self.text_index = text_index
        self.sent_index = sent_index
        self.word_index = word_index
        self.unify(morph)
        self.show = show
        self.lemma = lemma
        
    def unify(self, gram):
        info = CODE_POS.get(gram)
        self.pos = info['pos']
        self.if_gram(info)
        self.dep = info.get('dep')
        
    def if_gram(self, info):
        if 'gram' in info: self.gram = info['gram']
        else: self.gram = []
    
    def __str__(self):
         return self.show
        
    def __repr__(self):
         return self.show
    
    def __hash__(self):
        return hash(self.show)

    def __eq__(self, other):
        return self.show == other.show
    
    def __lt__(self, other):
        return self.show < other.show
    
    def __gt__(self, other):
        return self.show > other.show

In [21]:
cur.execute("""
SELECT token.lemma, morph, representation, text_id, sentence_id, idx 
FROM text_content
    JOIN token ON text_content.token_id = token.id
    JOIN ngram_item ON token.ngram_text_id = ngram_item.id
""")
sentences = []
text, sent = None, None
for lemma, morph, show, text_index, sent_index, word_index in cur:
    if text_index == text and sent_index == sent:
        pass
    else:
        sentences.append([])
        text, sent = text_index, sent_index
    sentences[-1].append(Word(lemma, morph, show, text_index, sent_index, word_index))

In [22]:
len(sentences)

101359

Всего в корпусе 101359 предложений.

In [23]:
# import pickle as pkl

# with open("sentences_lemma.pkl", "wb") as f:
#     pkl.dump(sentences, f)

### 2.2. Формирование нграмм и их фильтрация

Формирование нграмм осуществляется с помощью nltk. Возможная длина нграмм составляет от 4 до 8 слов. Каждая нграмма перед добавлением в общий список проверялась на соответвие требованиям, которые были выделены для повторяющихся конструкций в сагах: 
* обязательность наличия в конструкции глагола
* синтаксически цельность (см. README.md)
* частеречная значимость не менее 90% (см. README.md)

Если нграмма соответвует всем критериям, она добавляется в словари, хранящие их частотности и координаты. В связи с относительно свободным порядком слов в исландском языке, для возможности слопывания конструкций, отличающихся лишь порядом слов, все слова в конструкциях были остортированы по части речи. 

In [24]:
class Ngram_storage:
    """
    Ngram_storage is a class that provides methods for generating and filtering ngrams
    """

    def __init__(self, g=grinfo):
        self.g = g
        self.reboot()
        
    def reboot(self):
        """
        This function initializes empty data
        """
        self.pos = defaultdict(int)
        self.deps = []
        self.gram = []
        self.result = []
        self.n_words = 0
        
    def add_gram(self, w):
        """
        This function adds grammatical information
        """
        if w.gram: gr = (w.pos, w.gram)
        else: gr = w.pos
        self.gram.append(gr)
        
    def add_word(self, w):
        """
        This function adds word to ngram
        """
        self.result.append(w)
        self.pos[w.pos] += 1
        self.n_words += 1
        self.add_gram(w)
        if w.dep: 
            self.deps.append(w.dep)
        
    def ngram_weight(self):
        """
        This function culculates significance of the entire ngram
        based on the significance of its parts of speech
        """
        s_pos = [self.g.POS_values[wp] * self.pos[wp] for wp in self.pos]
        s_pos = sum(s_pos) / self.n_words
        if len(pos) == 1: 
            s_pos *= 0.8
        return s_pos
    
    def __check_ldeps(self, dep, found):
        """
        This function checks compound dependencies
        """
        for d in dep:
            if d in self.gram:
                self.gram.remove(d)
                found += 1
                break
        return found
    
    def __check_dependencies(self):
        """
        This function checks dependencies in a ngram
        """
        found = 0
        for dep in self.deps:
            if isinstance(dep, list):
                found = self.__check_ldeps(dep, found)
            elif dep in self.gram:
                self.gram.remove(dep)
                found += 1
        if found != len(self.deps):
            return False
        return True
    
    def ifngram(self):
        """
        This function checks if the word sequence is an ngram (=construction)
        """
        if 'V' in self.pos and self.n_words > 2: 
            dep = self.__check_dependencies()
            val = self.ngram_weight()
            if dep and val > 0.9: return True
        return False
        
    def clean_adj(self):
        """
        This function shortens the sequence of adjectives
        """
        res = []
        adj = False
        for w in self.result:
            if adj and w.pos != 'Adj': adj = False
            if not adj: res.append(w)
            if w.pos == 'Adj': adj = True
        res = sorted(res)
        self.result = tuple(res)
        
    def add_ngram(self, all_ngrams):
        """
        This function adds ngram to final list of ngrams
        """
        if self.ifngram():
            self.clean_adj()
            all_ngrams.add(self.result)
            self.reboot()

In [25]:
def ngram_maker(sent, start, finish, all_ngrams):
    
    if len(sent) < 4: start = len(sent)
    if len(sent) < 8: finish = len(sent)
        
    for num_w in range(start, finish):
        for ng in ngrams(sent, num_w):
            storage = Ngram_storage()
            
            for index, w in enumerate(ng):
                if w.lemma in skip_conj:
                    if index == 0 or index == len(ng)-1:
                        continue
#                 elif w.pos == 'pun' and w.lemma != ',':
#                     storage.add_ngram(all_ngrams)
                elif w.pos != 'pun': 
                    storage.add_word(w)
            if storage.result: 
                storage.add_ngram(all_ngrams)

In [26]:
all_ngrams = AllNgrams()
start, finish = 4, 8
skip_conj = ('og', 'en', 'eða')

for sent in tqdm(sentences):
    ngram_maker(sent, start, finish, all_ngrams)

In [28]:
len(all_ngrams.freq)

663775

В результате получилось 663775 конструкций разной степени вариативности.

In [29]:
nrams_sort = sorted(all_ngrams.freq.items(), key=lambda kv: kv[1])

In [30]:
nrams_sort[-15:]

[((PProNom, PrpNom, V_svarar), 378),
 ((C_er, NNom_maður, PrpNom, V_hét), 386),
 ((C_er, PProNom, V_komu), 414),
 ((PProDat, PrpNom, V_segir), 416),
 ((PProNom, PrpNom, V_mælti), 431),
 ((C_að, PProNom, PrpNom, V_segir), 441),
 ((PProNom, PProNom, V_segir), 481),
 ((C_er, PrpNom, V_hét), 580),
 ((C_að, PProNom, V_segir), 587),
 ((C_er, PrpNom, V_átti), 594),
 ((C_að, PrpNom, V_segir), 676),
 ((NNom_son, PProGen, PrpNom, V_var), 677),
 ((C_er, PrpNom, PrpNom, V_átti), 731),
 ((PProNom, PrpNom, V_segir), 860),
 ((ADV_þá, PrpNom, V_mælti), 1637)]

### 2.3. Фильтрация на основе частотности

Так как в ходе формирования нграмм рассматривались сразу несколько размеров окон, то часть нграмм представляется собой дублирование одного и того же предложения. Например, выражение <i>Og er Ásbirni komu orð bróður síns</i> может быть представлено в данных 4 раза:

    1) Og er Ásbirni komu - окно в 4 слова 
    2) Og er Ásbirni komu orð - окно в 5 слов
    3) Og er Ásbirni komu orð bróður - окно в 6 слов
    4) Og er Ásbirni komu orð bróður síns - окно в 7 слов

Для того, чтобы сократить подобное дублирование, все конструкции были сгруппированы по частотности. Если в одну группу попали конструкции, отличающиеся лишь словами в конце конструкции, значит, это варианты одной длинной конструкции, которая была продублирована окнами разной длины. Группировка по частотности позволяет избежать сокращения конструкций, встручающихся и в других предложениях. Так, если <i>Og er Ásbirni komu</i> встретиться сразу в нескольких предложениях, ее частотность будет выше, чем у остальных вариантов. Это отличие позволяет рассматривать <i>Og er Ásbirni komu</i> уже не как часть большей конструкции, но как самостоятельную единицу. 

In [31]:
def filter_variants(freq_grouped):
    unique = []
    ind, lead = 0, ''
    for group in freq_grouped:
        for index, item in enumerate(freq_grouped[group]):
            if index == 0 or item[:ind] == lead: 
                lead = item
                ind = len(item)
            elif item[:ind] != lead:
                unique.append(lead)
                lead = item
                ind = len(item) 
    return list(reversed(unique))

In [32]:
freq_grouped = defaultdict(list) # частотность: нграммы

for key, value in nrams_sort:
    freq_grouped[value].append(key)

In [33]:
len(freq_grouped)

209

In [34]:
unique_ngrams = filter_variants(freq_grouped)

После того, как конструкции были отфильтрованы, они были записаны в базу.

In [35]:
cur.execute("SELECT representation, id FROM ngram_item")
token_mapping = dict(cur.fetchall())

In [36]:
ngram_to_db = []
content_to_db = []
coords_to_db = []
for idx, ng in enumerate(unique_ngrams):
    ngram_to_db.append((idx, all_ngrams.freq[ng]))
    for word in ng:
        content_to_db.append((idx, token_mapping[word.show]))
    for coord in all_ngrams.coord[ng]:
        coords_to_db.append((idx, coord.text_index, coord.sent_index, coord.word_start, coord.word_end))

In [37]:
cur.executemany("INSERT INTO ngram_coords (unique_ngram_id, text_id, sentence_id, start, end) VALUES (?, ?, ?, ?, ?)", coords_to_db)
db.commit()

In [38]:
cur.executemany("INSERT INTO ngram_content (ngram_id, ngram_item_id) VALUES (?, ?)", content_to_db)
db.commit()

In [39]:
cur.executemany("INSERT INTO unique_ngrams (id, n_occurrences) VALUES (?, ?)", ngram_to_db)
db.commit()

## 3. Упрощение нграмм

После первичной фильтрации и схлопывания, было произведено упрощение нграмм, которое заключалось в следующих этапах:

    1) формирование "коротких представления" нграмм (нграммы с опущенными прилагательными и наречиями)
    2) формирование "синтаксического представления" нграмм за счет опущения лемм
    
Этот этап необходим для дальнейшей кластеризации. На основе получившихся упрощенных представлений нграммы группировались в наборы, в рамках которых производилась кластеризация, что позволяет находить расстояния только между схожими синтаксическими конструкциями.  


short_ngram - это без прилагательных и наречий

pos_ngrams - это только с метками

In [40]:
cur.execute("SELECT id FROM ngram_item WHERE pos NOT IN ('', 'ADV', 'Adj')")
good = cur.fetchall()

Создать индекс по ngram_item_id

In [41]:
cur.execute("""
CREATE INDEX `ngram_item_id_index` ON `ngram_content` (
	`ngram_item_id`	ASC
);
""")
db.commit()

In [42]:
cur.executemany("UPDATE ngram_content SET keep = 1 WHERE ngram_item_id = ?", good)
db.commit()

In [43]:
cur.execute("UPDATE ngram_content SET keep = 0 WHERE keep IS NULL")
db.commit()

In [44]:
cur.execute("""
SELECT ngram_id, kind
FROM ngram_content
    JOIN ngram_item ON ngram_content.ngram_item_id = ngram_item.id
WHERE keep = 1
ORDER BY ngram_id, kind""")
data = cur.fetchall()

ngram_dict = {}
short_ngram_list = []
ngram_update = []
ngram_idx = 0

current_ngram = tuple()
current = 0
for row in tqdm(data, leave=False):
    if row[0] != current:
        if current_ngram:
            if current_ngram not in ngram_dict:
                ngram_dict[current_ngram] = ngram_idx
                short_ngram_list.append((ngram_idx, ';'.join(current_ngram)))
                ngram_idx += 1
            ngram_update.append((ngram_dict[current_ngram], current))
            current = row[0]
            current_ngram = (row[1],)
    else:
        current_ngram += (row[1],)

In [45]:
current_ngram

('NNom', 'PProNom', 'PrpNom', 'V')

In [46]:
len(ngram_dict), len(short_ngram_list), len(ngram_update)

(12747, 12747, 658132)

In [47]:
cur.executemany("UPDATE unique_ngrams SET short_ngram_id = ? WHERE id = ?", ngram_update)
db.commit()

In [48]:
short_ngram_list[:5]

[(0, 'C;PProNom;PProNom;V'),
 (1, 'C;PProDat;PProNom;V'),
 (2, 'PProNom;V'),
 (3, 'NNom;PProGen;PrpNom;V'),
 (4, 'C;V;V')]

In [49]:
cur.executemany("INSERT INTO short_ngrams (id, representation) VALUES (?, ?)", short_ngram_list)
db.commit()

In [50]:
cur.execute("""
SELECT count(ngram_coords.id), unique_ngrams.short_ngram_id 
FROM ngram_coords
	JOIN unique_ngrams ON ngram_coords.unique_ngram_id = unique_ngrams.id
WHERE unique_ngrams.short_ngram_id IS NOT NULL
GROUP BY unique_ngrams.short_ngram_id""")
data = cur.fetchall()

cur.executemany("UPDATE short_ngrams SET n_occurrences = ? WHERE id = ?", data)
db.commit()

## 4. Кластеризация

Выбираем 1 pos_ngram

Подтягиваем нграм-контент keep

Оправляем в кластеризацию

In [3]:
class Clusters:
    
    def __init__(self, model_path):
#         self.load_model(model_path)
        self.coefs = {'V': 3, 'PREP': 2, 'PProNom': 1.5, 'Inf': 1.5, 'C': 2}
        
    def load_model(self, model_path):
        """
        Load model
        """
        self.model = load_facebook_vectors(model_path)
    
    def __parse_single(self, w_list, coef):
        """
        Compute similarities
        """
        return abs(1 - cosine_similarity(self.model[w_list])) * coef
    
    def __parse_double(self, w_list1, w_list2, coef):
        """
        Align double pos-tags and compute similarities
        """
        w_list1 = self.model[w_list1]
        w_list2 = self.model[w_list2]

        A = cosine_similarity(w_list1)
        B = cosine_similarity(w_list2)
        C = cosine_similarity(w_list1, w_list2)
        D = C.T

        Z = np.concatenate([A.flatten(), B.flatten(), C.flatten(), D.flatten()])
        Z = Z.reshape((4, A.flatten().shape[0]))
        M = Z.argmax(axis=0)

        R = ((Z[0:2,] * (M <= 1)) + (Z[2:4,] * (M > 1))).sum(axis=0).reshape(A.shape)
        return abs(2 - R) * coef
    
    @staticmethod
    def __get_sentences(short_id):
        """
        Select ngram content for chosen type
        """
        cur.execute("""
        SELECT ngram_id, lemma, kind 
        FROM ngram_content
            JOIN ngram_item ON ngram_content.ngram_item_id = ngram_item.id
            JOIN unique_ngrams ON ngram_content.ngram_id = unique_ngrams.id
        WHERE short_ngram_id = ? AND keep = 1
        """, (short_id,))
        return cur.fetchall()
    
    @staticmethod
    def __df_for_unique(data):
        """
        Create df for unique structure (C, V)
        """
        df = pd.DataFrame(data, columns=["ngram_id", "lemma", "kind"])
        df["lemma"] = df[["lemma", "kind"]].apply(lambda x: x["lemma"] or x["kind"], axis=1)
        df = df.pivot_table(index="ngram_id", columns="kind", values="lemma", aggfunc='first')
        return df
    
    @staticmethod
    def __df_for_non_unique(phrases_list):
        """
        Create df for non-unique structure (C, V, V)
        """
        data = []
        idxs = []
        
        current_ngram = phrases_list[0][0]
        current_row = {}
        
        for idx, row in enumerate(phrases_list):
            ng_id, lemma, kind = row
            lemma_to_write = lemma or kind
            
            if ng_id != current_ngram:
                data.append(current_row)
                idxs.append(current_ngram)
                current_row = {}
                current_ngram = ng_id
            
            if kind in current_row:
                current_row[f"{kind}_2"] = lemma_to_write
            else:
                current_row[kind] = lemma_to_write
        data.append(current_row)
        idxs.append(current_ngram)
        df = pd.DataFrame(data)
        cols = df.columns.tolist()
        df.index = idxs
        return df
    
    
    def __make_dataframe(self, short_id, key):
        """
        Create dataframe depending on ngram type
        """
        data = self.__get_sentences(short_id)
        if max(Counter(key.split(";")).values()) == 1:
            df = self.__df_for_unique(data)
        else:
            df = self.__df_for_non_unique(data)
        return df
        
    
    def __get_distance_matrix(self, key, df):
        """
        Compute distance matrix
        """
        poss = Counter(key.split(";"))
        matrix = np.zeros((df.shape[0], df.shape[0]))
        for pos in poss:
            coef = self.coefs.get(pos, 1)
            if poss[pos] == 2:
                matrix = np.add(matrix, self.__parse_double(df[pos].values, df[f"{pos}_2"].values, coef=coef), out=matrix)
            elif poss[pos] == 1:
                matrix = np.add(matrix, self.__parse_single(df[pos].values, coef=coef), out=matrix)
        matrix = matrix / len(poss)
        return matrix
    
    @staticmethod
    def __validate_key(representation):
        """
        Validate ngram type: max 2 same pos-tags
        """
        return max(Counter(representation.split(";")).values()) <= 2
    
    def get_keys(self):
        """
        Select all valid ngram types
        """
        cur.execute("""
        SELECT id, representation FROM short_ngrams
        WHERE n_occurrences > 10""")
        data = [i for i in cur.fetchall() if self.__validate_key(i[1])]
        return data
        
    def __run_one(self, short_id, key):
        """
        Run clusterization for one ngram type
        """
        df = self.__make_dataframe(short_id, key)
        
        M = self.__get_distance_matrix(key, df)
        Z = complete(M)
        C = fcluster(Z, t=1, criterion="distance")
        
        cntr = {k: v for k, v in Counter(C).items() if v > 1} 
        cluster_result = pd.DataFrame({"ids": df.index, "clusters": C, "short_id": short_id})
        cluster_result = cluster_result[cluster_result["clusters"].isin(cntr)]
        
        cur.executemany(
            """INSERT INTO clustering (ngram_id, short_ngram_id, cluster_idx)VALUES (?, ?, ?)""", 
            cluster_result[["ids", "short_id", "clusters"]].values.tolist())
        db.commit()
        
        
    def run(self):
        """
        Run clusterization on all ngrams
        """
        keys = self.get_keys()  #[1000:]  # - первые тяжелые, проще тестить на поменьше
        for idx, short_form in tqdm(keys, leave=False):
            self.__run_one(idx, short_form)

In [4]:
# тут не через класс загрузка модели, потому что так проще тестировать, а то модель загружается долго и тяжело

model_path = '/home/dkbrz/data/cc.is.300.bin'

model = load_facebook_vectors(model_path)

In [5]:
clusters = Clusters(model_path)

In [6]:
clusters.model = model

In [ ]:
clusters.run()